S16 (). ¿Cuál es el usuario más quejumbroso por estado? Consideramos quejumbroso a un usuario que tiene más de 5 reviews y son todas de 2 estrellas o menos.

# Carga de archivos


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1f5d423b4700284557eefd468252ec977ad0d0455e298b9854a2f6aca151a7e4
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!apt install openjdk-8-jdk-headless -qq

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime as dt

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1Unn0g_Kw2aEpvrmqJGBrrki9_YzWsEJG"})   # replace the id with id of file you want to access
downloaded.GetContentFile('review.csv') 

In [ ]:
downloaded = drive.CreateFile({'id':"14sjAJe8wBi7N_1xxJFCm7Mtzrm_gFiPj"})   # replace the id with id of file you want to access
downloaded.GetContentFile('business.csv') 

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
dfReview = (spark.read\
      .option("multiline", "true")\
      .option("quote", '"')\
      .option("header", "true")\
      .option("escape", "\\")\
      .option("escape", '"')\
      .csv('review.csv'))\
      .select(['business_id', 'user_id', 'stars'])\
      .dropna()
rddReview = dfReview.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
sqlContext = SQLContext(sc)
dfBusiness = (spark.read\
      .option("multiline", "true")\
      .option("quote", '"')\
      .option("header", "true")\
      .option("escape", "\\")\
      .option("escape", '"')\
      .csv('business.csv'))\
      .select(['business_id', 'state'])\
      .dropna()
rddBusiness = dfBusiness.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


# Calculo

In [ ]:
rddB = rddBusiness.map(lambda x: (x[0], x[1])) #(business_id, state)


Lo que hago aca es:\
1 - hago un map para cada review donde pongo como clave el user_id y valores una lista [boolean, 1, [business_id]] donde el boolean es True o false segun si la votacion no supera las 3 estrellas

2- en el reduceByKey lo que hago es, para el booleano le aplico and para ver si todas las votaciones son menores a 3, el 1 lo sumo para obtener las n reviews del user, y el business_id lo sumo para generar una lista de business_id

3- filtro para asegurarme que todas las reviews son menores a 3 y ademas me fijo que tenga mas de 5



In [ ]:
usersQuejosos = rddReview.map(lambda x: (x[1], [True, 1, [x[0]]]) if float(x[2]) < 3 else (x[1], [False, 1, [x[0]]]))\
                .reduceByKey(lambda x,y: [x[0] and y[0], x[1]+y[1], x[2]+y[2]] )\
                .filter(lambda x: x[1][0] and (x[1][1] > 5 ) )

Aca lo que hago es hacer un flatmap para obtener como keys los business id, y como values el user_id

In [ ]:
negociosMalVotados = usersQuejosos.flatMap(lambda x:  [(business,[x[0],x[1][1]]) for business in x[1][2] ])

Hago esto para poder joinear y asi obtener el estado de que es cada negocio, una vez hecho el join, lo remapeo para que la key sea el estado, y los valores el user_id, y la sumatoria de reviews

In [ ]:
joined = rddB.join(negociosMalVotados)            

Por ultimo hago un reduceByKey para obtener los maximos de cada key (estado)

In [ ]:
masQuejumbrosos = joined.map(lambda x: (x[1][0], [x[1][1][0], x[1][1][1]]))\
                        .reduceByKey(lambda x,y: x if x[1]>y[1] else y)

Con un collect muestro todos los rdd (puedo usar collect porque se que en el peor de los casos son 50 estados)

In [ ]:
masQuejumbrosos.collect()

[('MO', ['_HGW2i7EdvkKH04JtByGFQ', 32]),
 ('TN', ['HYKIOKPr2KQbk-72V4ANVg', 21]),
 ('LA', ['J3hkjFPnMIPHQe_PuSJ36Q', 16]),
 ('CA', ['gSbWYsI9D_CLUsheOo_ttw', 10]),
 ('DE', ['T3oldmX-JGVMql9GjEAEMg', 23]),
 ('PA', ['ttyzgc93vTrwIYoDY0UmXw', 24]),
 ('FL', ['2uyyLfTPfF9vckrg5SHS6g', 32]),
 ('IN', ['CLg0CzbxOK1qfvzG6rP_Zg', 13]),
 ('NV', ['8eVcgYY-gMJDKODwn2lSZw', 22]),
 ('AZ', ['qpmzZ5G7x-IMSTb2VYeJ_Q', 28]),
 ('NJ', ['h_kxXndPx1jLV9P4WzekTA', 15]),
 ('ID', ['KBSglJcN61YIWleS04TJtQ', 14]),
 ('IL', ['_HGW2i7EdvkKH04JtByGFQ', 32]),
 ('AB', ['A2KEoDZ7qqO_SDImvoFJrw', 9])]